In [144]:
from manim import *
from common import nuclear
from importlib import reload
reload(nuclear)

<module 'common.nuclear' from '/home/mic/Documents/code/python/boldNotes/common/nuclear.py'>

In [146]:
%%manim -v WARNING --disable_caching -qm MainScene

config.media_width = "75%"
config.media_embed = True

class MainScene(Scene):
    def construct(self):
        nucleus = nuclear.create_nucleus(6, 6, 3, 0.6, seed=1, num_tries=50)
        nucleus.move_to(0)
        self.play(Create(nucleus))  # show the circle on screen
        self.wait(3)